In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)



!pip install kaggle


Download 100%.
    100% |████████████████████████████████| 61kB 4.2MB/s 
    100% |████████████████████████████████| 245kB 11.6MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/5a/2d/0c/9fc539e558586b9ed9127916a7f4e620163c24cc97460b1188
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify


In [0]:
# if not os.path.exists('./data'):
!cp /content/.kaggle/kaggle.json /root/.kaggle/kaggle.json
!kaggle competitions download -c humpback-whale-identification


  0% 0.00/498k [00:00<?, ?B/s]
100% 498k/498k [00:00<00:00, 78.4MB/s]
  0% 0.00/594k [00:00<?, ?B/s]
100% 594k/594k [00:00<00:00, 93.9MB/s]
 99% 1.34G/1.35G [00:11<00:00, 127MB/s]
100% 1.35G/1.35G [00:11<00:00, 128MB/s]
100% 4.14G/4.16G [00:30<00:00, 118MB/s] 
100% 4.16G/4.16G [00:30<00:00, 148MB/s]


In [0]:
#!ls -a /root/.kaggle

#!cp /content/.kaggle/kaggle.json /root/.kaggle/kaggle.json
if not os.path.exists('./train'):
  !unzip -q train.zip -d train

!ls train | wc -l

if not os.path.exists('./test'):
  !unzip -q test.zip -d test

!ls test | wc -l

25361
7960


# New Section

In [0]:
image_paths = list()
for (dirpath, dirnames, filenames) in os.walk("train"):
    for file in filenames:
        if '.jpg' in file and not file.startswith('.'):
              image_paths.append(os.path.join(dirpath, file))

